In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
from anchor import anchor_tabular
from int_met import *

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
dataset_folder = 'Datasets/'
dataset = utils.load_dataset('mortality', balance=True, dataset_folder=dataset_folder)

In [3]:
explainer = anchor_tabular.AnchorTabularExplainer(dataset.class_names, dataset.feature_names, dataset.data, dataset.categorical_names)
explainer.fit(dataset.train, dataset.labels_train, dataset.validation, dataset.labels_validation)

In [4]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5, random_state= 46)
c.fit(explainer.encoder.transform(dataset.train), dataset.labels_train)
predict_fn = lambda x: c.predict(explainer.encoder.transform(x))
print('Train', sklearn.metrics.accuracy_score(dataset.labels_train, predict_fn(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(dataset.labels_test, predict_fn(dataset.test)))
print('Train', sklearn.metrics.roc_auc_score(dataset.labels_train, predict_fn(dataset.train)))
print('Test', sklearn.metrics.roc_auc_score(dataset.labels_test, predict_fn(dataset.test)))

Train 0.9754784688995215
Test 0.8169856459330144
Train 0.9755413550207725
Test 0.8165940366972477


In [5]:
exp_fn_bulk = lambda x: np.array([np.array(explainer.explain_instance(x[idx], c.predict, threshold=0.95).names()) for idx in range(len(x))])

In [6]:
temp = dataset.test[:412]
unique_data, unique_idx = np.unique(temp,axis=0,return_index=True)
unique_label = dataset.labels_test[unique_idx]
unique_data.shape

(399, 14)

In [7]:
%time exp2 = exp_fn_bulk(unique_data[:10])

Wall time: 1min 28s


In [8]:
%time exp2 = exp_fn_bulk(unique_data[:10])

Wall time: 1min 31s


In [9]:
%time exp2 = exp_fn_bulk(unique_data[:10])

Wall time: 1min 33s


In [ ]:
exp1

In [ ]:
%time exp1 = exp_fn_bulk(unique_data)
%time exp2 = exp_fn_bulk(unique_data)

In [16]:
i = calc_identity(exp1,exp2)
print(i)

(89.25, 43, 400)


In [22]:
s = calc_separability(exp1)
print (s)

(102, 400, 160000, 0.06375)


In [23]:
from copy import deepcopy, copy
def get_cat_names(cat_feats, cate_names, feat_names):
    cat_names = deepcopy(cate_names)
    for i in cat_feats:
        for j in range(len(cat_names[i])):
            cat_names[i][j] = feat_names[i] + ' = ' + cat_names[i][j]
    return cat_names

In [24]:
cat_names = get_cat_names(list(set(dataset.categorical_features) - set(dataset.ordinal_features)), dataset.categorical_names, dataset.feature_names)
flat_cat_names = [item for sublist in cat_names.values() for item in sublist]

In [25]:
def encode_exp(exp, flat_cat_names):
    enc_exp = np.zeros((1,len(flat_cat_names)))
    for i in range(len(flat_cat_names)):
        if flat_cat_names[i] in exp:
            enc_exp[0,i] = 1
    return enc_exp

In [26]:
enc_exps = np.array([encode_exp(exp1[i], flat_cat_names) for i in range(len(exp1))]).squeeze()
enc_exps2 = np.array([encode_exp(exp2[i], flat_cat_names) for i in range(len(exp2))]).squeeze()

In [31]:
sb = calc_stability(enc_exps, unique_label)
print(sb)

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


(104, 400)
